Drive Link for sample data: https://drive.google.com/file/d/1R6HOBJc2368229_BsWBLV0-SbMQlpeIl/view?usp=sharing

In [ ]:
import os
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.models as models
from PIL import Image
from glob import glob
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
import pickle as pkl
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.utils.rnn import pack_padded_sequence

%matplotlib inline

In [2]:
df = pd.read_csv('questiondf.csv',index_col=0)
df.head()

,image_id,question,question_id
0,458752,What is this photo taken looking through?,458752000
1,458752,What position is this man playing?,458752001
2,458752,What color is the players shirt?,458752002
3,458752,Is this man a professional baseball player?,458752003
4,262146,What color is the snow?,262146000


In [3]:
image_IDs = list(set(df['image_id']))
len(image_IDs)

1919

In [14]:
#generate filenames from image ids
# for i in ids:
#     name=str(i)
#     l = len(name)
#     filename=''.join(['0'for x in range(12-l)])+name+".jpg"
#     print(filename)

In [4]:
#check questions corresponding to an image id
# df[df['image_id']==49]
# sorted(image_IDs)

In [5]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
device = torch.device("cuda" if use_cuda else "cpu")

CUDA is not available.  Training on CPU ...


In [6]:
# To initialise dataset
# from gluoncv import data, utils
# from matplotlib import pyplot as plt

# train_dataset = data.COCODetection(splits=['instances_train2017'])
# val_dataset = data.COCODetection(splits=['instances_val2017'])
# print('Num of training images:', len(train_dataset))
# print('Num of validation images:', len(val_dataset))

In [7]:
total_files = np.array(glob("/Users/inika/Desktop/Shortcuts/mini/Visual-Question-Answering/Data/train2017/*"))
print('There are %d total images.' % len(total_files))

There are 118287 total images.


In [16]:
images = []
ids = []
img_paths = []
for f in total_files:
    img_id = int(f.split("/")[-1].split(".")[0])
    if img_id in image_IDs:
        img = Image.open(f)
        if img.mode=='RGB':
            images.append(img)
            ids.append(img_id)
            img_paths.append(f)

In [21]:
# dst = '/Users/inika/Desktop/Shortcuts/mini/Visual-Question-Answering/Data/train_sample'
# from shutil import copy
# for f in img_paths:
#     copy(f, dst)

In [9]:
# for i in range(len(images)):
#     if images[i].mode!='RGB':
#         print(i,images[i].mode) # 8-bit pixels, black and whi

In [22]:
from img2vec_pytorch import Img2Vec
from PIL import Image

img2vec = Img2Vec(cuda=False)

/usr/local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:219: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [ ]:
vectors = img2vec.get_vec(images)